# Zusammenfügen der relevanten Daten

**Differenzen berechnen**

Im folgenden Abschnitt werden die unterschiedlichen TTR Differenzen berechnet. Hierbei wird in Folgenden Abständen differenziert:

Ein **Jahres** Differenz:
- 2024 Q1 - 2025 Q1

**Halbjährliche** Differenz:
- 2024 Q1 - 2024 Q3

**Quartalsweise** Differenz:
- 2024 Q1 - 2024 Q2
- 2024 Q2 - 2024 Q3
- 2024 Q3 - 2024 Q4
- 2024 Q4 - 2025 Q1

In [6]:
import duckdb
import pandas as pd
import os

# Pfade zu den CSV-Dateien definieren
data_dir = "/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/csv_Daten"
output_dir = "/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/processed_data"

# Sicherstellen, dass der Ausgabeordner existiert
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Dateinamen der Quelldateien
files = {
    "Q1_2024": "Q-TTR-Liste_20240211_komplett.csv",
    "Q2_2024": "Q-TTR-Liste_20240511_komplett.csv",
    "Q3_2024": "Q-TTR-Liste_20240811_komplett.csv",
    "Q4_2024": "Q-TTR-Liste_20241211_komplett.csv",
    "Q1_2025": "Q-TTR-Liste_20250211_komplett.csv"
}

# DuckDB-Verbindung initialisieren
conn = duckdb.connect(database=':memory:')

# Tabellen in DuckDB erstellen
for period, file in files.items():
    file_path = os.path.join(data_dir, file)
    table_name = f"ttr_{period.lower()}"
    
    # CSV einlesen mit ALL_VARCHAR=1, um alle Spalten als Text zu behandeln
    try:
        conn.execute(f"""
            CREATE TABLE {table_name} AS 
            SELECT * FROM read_csv_auto(
                '{file_path}', 
                header=TRUE, 
                delim=',',
                ALL_VARCHAR=1
            );
        """)
        
        # Überprüfen, wie viele Zeilen eingelesen wurden
        row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
        print(f"Eingelesene Zeilen für {period}: {row_count}")
        
    except Exception as e:
        print(f"Fehler beim Einlesen von {file}: {e}")

# Standardisiere die Tabellen mit korrekten Typen für die wichtigen Spalten
for period in files.keys():
    table_name = f"ttr_{period.lower()}"
    std_table_name = f"std_{table_name}"
    
    try:
        # Prüfen ob GeschlechtImSpielbetrieb oder Geschlecht existiert
        column_info = conn.execute(f"PRAGMA table_info({table_name})").fetchdf()
        has_geschlecht_im_spielbetrieb = "GeschlechtImSpielbetrieb" in column_info['name'].tolist()
        
        gender_column = "GeschlechtImSpielbetrieb" if has_geschlecht_im_spielbetrieb else "Geschlecht"
        
        conn.execute(f"""
            CREATE TABLE {std_table_name} AS
            SELECT 
                "InterneNr",
                "Nachname",
                "Vorname",
                "{gender_column}" AS "Geschlecht",
                "Geburtsdatum",
                "Nation",
                CAST("Einstufungswert zuletzt" AS FLOAT) AS "Einstufungswert zuletzt"
            FROM {table_name};
        """)
        
        print(f"Standardisierte Tabelle für {period} erstellt.")
    except Exception as e:
        print(f"Fehler bei der Standardisierung von {period}: {e}")

# Spieler finden, die in allen Datensätzen vorhanden sind
common_players_query = """
WITH common_players AS (
    SELECT "InterneNr" 
    FROM std_ttr_q1_2024
    INTERSECT
    SELECT "InterneNr" 
    FROM std_ttr_q2_2024
    INTERSECT
    SELECT "InterneNr" 
    FROM std_ttr_q3_2024
    INTERSECT
    SELECT "InterneNr" 
    FROM std_ttr_q4_2024
    INTERSECT
    SELECT "InterneNr" 
    FROM std_ttr_q1_2025
)
SELECT COUNT(*) FROM common_players
"""
common_count = conn.execute(common_players_query).fetchone()[0]
print(f"Anzahl Spieler, die in allen Datensätzen vorkommen: {common_count}")

# Erstellen des kombinierten Datensatzes mit den Differenzberechnungen
combined_data_query = """
CREATE TABLE combined_data AS
SELECT 
    q1_24."InterneNr",
    q1_24."Nachname",
    q1_24."Vorname",
    q1_24."Geschlecht",
    q1_24."Geburtsdatum",
    q1_24."Nation",
    q1_24."Einstufungswert zuletzt" AS "TTR_Q1_2024",
    q2_24."Einstufungswert zuletzt" AS "TTR_Q2_2024",
    q3_24."Einstufungswert zuletzt" AS "TTR_Q3_2024",
    q4_24."Einstufungswert zuletzt" AS "TTR_Q4_2024",
    q1_25."Einstufungswert zuletzt" AS "TTR_Q1_2025",
    q1_25."Einstufungswert zuletzt" - q1_24."Einstufungswert zuletzt" AS "Q-TTR_dif_24_Q1-25_Q1",
    q3_24."Einstufungswert zuletzt" - q1_24."Einstufungswert zuletzt" AS "Q-TTR_dif_24_Q1-24_Q3",
    q2_24."Einstufungswert zuletzt" - q1_24."Einstufungswert zuletzt" AS "Q-TTR_dif_24_Q1-24_Q2",
    q3_24."Einstufungswert zuletzt" - q2_24."Einstufungswert zuletzt" AS "Q-TTR_dif_24_Q2-24_Q3",
    q4_24."Einstufungswert zuletzt" - q3_24."Einstufungswert zuletzt" AS "Q-TTR_dif_24_Q3-24_Q4",
    q1_25."Einstufungswert zuletzt" - q4_24."Einstufungswert zuletzt" AS "Q-TTR_dif_24_Q4-25_Q1"
FROM 
    std_ttr_q1_2024 q1_24
JOIN 
    std_ttr_q2_2024 q2_24 ON q1_24."InterneNr" = q2_24."InterneNr"
JOIN 
    std_ttr_q3_2024 q3_24 ON q1_24."InterneNr" = q3_24."InterneNr"
JOIN 
    std_ttr_q4_2024 q4_24 ON q1_24."InterneNr" = q4_24."InterneNr"
JOIN 
    std_ttr_q1_2025 q1_25 ON q1_24."InterneNr" = q1_25."InterneNr"
"""

try:
    conn.execute(combined_data_query)
    combined_count = conn.execute("SELECT COUNT(*) FROM combined_data").fetchone()[0]
    print(f"Anzahl Einträge im kombinierten Datensatz: {combined_count}")

    # Speichern des kombinierten Datensatzes als CSV
    output_file = os.path.join(output_dir, "TTR_Entwicklung_Q1_2024_bis_Q1_2025.csv")
    conn.execute(f"""
        COPY combined_data TO '{output_file}' (HEADER, DELIMITER ',');
    """)

    print(f"Kombinierter Datensatz wurde gespeichert unter: {output_file}")

    # Statistiken und Top-Spieler
    if combined_count > 0:
        stats_query = """
        SELECT 
            AVG("Q-TTR_dif_24_Q1-25_Q1") AS "Durchschnittliche Änderung über 1 Jahr",
            MIN("Q-TTR_dif_24_Q1-25_Q1") AS "Größter Abfall über 1 Jahr",
            MAX("Q-TTR_dif_24_Q1-25_Q1") AS "Größter Anstieg über 1 Jahr",
            STDDEV("Q-TTR_dif_24_Q1-25_Q1") AS "Standardabweichung der Änderung"
        FROM combined_data
        """
        stats = conn.execute(stats_query).fetchdf()
        print("\nStatistiken zur TTR-Entwicklung über ein Jahr:")
        print(stats)

        top_improvers_query = """
        SELECT 
            "Nachname", 
            "Vorname", 
            "TTR_Q1_2024", 
            "TTR_Q1_2025", 
            "Q-TTR_dif_24_Q1-25_Q1" AS "Anstieg"
        FROM combined_data
        ORDER BY "Q-TTR_dif_24_Q1-25_Q1" DESC
        LIMIT 10
        """
        top_improvers = conn.execute(top_improvers_query).fetchdf()
        print("\nTop 10 Spieler mit dem größten TTR-Anstieg:")
        print(top_improvers)
except Exception as e:
    print(f"Fehler bei der Erstellung des kombinierten Datensatzes: {e}")
finally:
    # Schließe die Verbindung
    conn.close()

Eingelesene Zeilen für Q1_2024: 754592
Eingelesene Zeilen für Q2_2024: 758001
Eingelesene Zeilen für Q3_2024: 762570
Eingelesene Zeilen für Q4_2024: 777605
Eingelesene Zeilen für Q1_2025: 781351
Standardisierte Tabelle für Q1_2024 erstellt.
Standardisierte Tabelle für Q2_2024 erstellt.
Standardisierte Tabelle für Q3_2024 erstellt.
Standardisierte Tabelle für Q4_2024 erstellt.
Standardisierte Tabelle für Q1_2025 erstellt.
Anzahl Spieler, die in allen Datensätzen vorkommen: 754474
Anzahl Einträge im kombinierten Datensatz: 754474
Kombinierter Datensatz wurde gespeichert unter: /Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/processed_data/TTR_Entwicklung_Q1_2024_bis_Q1_2025.csv

Statistiken zur TTR-Entwicklung über ein Jahr:
   Durchschnittliche Änderung über 1 Jahr  Größter Abfall über 1 Jahr  \
0                                1.557188                      -611.0   

   Größter Anstieg über 1 Jahr  Standardabweichung der Änderung  
0                        493.0 

## Entfernen der irrelevanten Spalten

Hierbei werden alle spalten entfernt die in jeder dif berechnung ein 0 eintrag enthalten

In [7]:
import pandas as pd
import duckdb

# Datei einlesen
TTR_Entwicklung_df = pd.read_csv("/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/processed_data/TTR_Entwicklung_Q1_2024_bis_Q1_2025.csv")

# Liste der zu überprüfenden Spalten
spalten_zu_pruefen = [
    "Q-TTR_dif_24_Q1-25_Q1", 
    "Q-TTR_dif_24_Q1-24_Q3", 
    "Q-TTR_dif_24_Q1-24_Q2", 
    "Q-TTR_dif_24_Q2-24_Q3", 
    "Q-TTR_dif_24_Q3-24_Q4", 
    "Q-TTR_dif_24_Q4-25_Q1"
]

# DuckDB-Verbindung herstellen
con = duckdb.connect(database=':memory:')

# DataFrame in DuckDB registrieren
con.register('ttr_df', TTR_Entwicklung_df)

# Erstellen der WHERE-Bedingung: Mindestens eine der Spalten muss ungleich 0 sein
bedingungen = [f'"{spalte}" != 0' for spalte in spalten_zu_pruefen]
where_bedingung = " OR ".join(bedingungen)

# SQL-Abfrage zum Filtern der Zeilen
query = f"""
SELECT *
FROM ttr_df
WHERE {where_bedingung}
"""

# Ausführen der Abfrage und Ergebnis als DataFrame speichern
gefilterter_df = con.execute(query).df()

print(f"Ursprüngliche Zeilenanzahl: {len(TTR_Entwicklung_df)}")
print(f"Gefilterte Zeilenanzahl: {len(gefilterter_df)}")
print(f"Entfernte Zeilen: {len(TTR_Entwicklung_df) - len(gefilterter_df)}")

# Optionales Speichern
# gefilterter_df.to_csv("TTR_Entwicklung_gefiltert.csv", index=False)

Ursprüngliche Zeilenanzahl: 754474
Gefilterte Zeilenanzahl: 34379
Entfernte Zeilen: 720095


In [8]:
gefilterter_df.to_csv("/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/processed_data/TTR_Entwicklung_ohne_nall.csv", index=False)

---

## **Hinzufügen der weiteren Daten**

In [3]:
import pandas as pd
import duckdb

TTR_Entwicklung_inport = pd.read_csv("/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/processed_data/TTR_Entwicklung_ohne_nall.csv")

# Pfad zur vollständigen QTTR-Liste
qttr_full = pd.read_csv("/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/full_data/Q-TTR-Liste_20240211_komplett.csv")

# DuckDB-Verbindung initialisieren
con = duckdb.connect(database=':memory:')

# Registriere die DataFrames als Tabellen in DuckDB
con.register('ttr_entwicklung', TTR_Entwicklung_inport)
con.register('qttr_liste', qttr_full)

# SQL-Abfrage zum Zusammenführen der Daten
# Hier nehmen wir alle Spalten aus ttr_entwicklung und
# fügen die zusätzlichen Spalten aus qttr_liste hinzu,
# wobei wir doppelte Spalten (die in beiden Tabellen vorkommen) vermeiden
sql_query = """
SELECT 
    t.*,
    q.Nr, q.Rang, q."Deutscher Rang", 
    q.GeschlechtImSpielbetrieb, 
    q.A, q."Einstufungsart zuletzt", q."Einstufungsdatum zuletzt", 
    q."Einstufungswert zuletzt", q."Einstufungsgruppe zuletzt", 
    q."Einstufungsposition zuletzt", q."Anzahl Einzel gesamt", 
    q.TTR, q."Kumulierte Inaktivitätsabzüge", q."Kumulierte QNSZ-Werte", 
    q."Kumulierte QNEB-Werte", q."Letztes Spiel", q.Verstorben, 
    q.Status, q."2J", q.lebt, q.MM, q.spb, q."10E", 
    q.Typ, q.Verband, q.Bezirk, q.Kreis, q.Verein, 
    q.Altersklasse, q.Spielklasse, q.Spielposition
FROM 
    ttr_entwicklung AS t
LEFT JOIN 
    qttr_liste AS q
ON 
    t.InterneNr = q.InterneNr
"""

# Abfrage ausführen und Ergebnis in neuem DataFrame speichern
merged_df = con.execute(sql_query).fetchdf()

# Prüfen, wie viele Zeilen erfolgreich zusammengeführt wurden
total_rows = len(TTR_Entwicklung_inport)
matched_rows = merged_df['Nr'].notna().sum()
match_percentage = (matched_rows / total_rows) * 100

print(f"Zusammenführung abgeschlossen.")
print(f"Gesamtzahl der Zeilen in TTR_Entwicklung_inport: {total_rows}")
print(f"Erfolgreich zugeordnete Zeilen: {matched_rows} ({match_percentage:.2f}%)")
print(f"Spalten im zusammengeführten DataFrame: {len(merged_df.columns)}")

# Überprüfen, ob Duplikate in der InterneNr-Spalte existieren
duplicate_count = merged_df.duplicated(subset=['InterneNr']).sum()
print(f"Anzahl der Duplikate nach InterneNr: {duplicate_count}")


merged_df.to_csv("/Users/fabi/Documents/BBA/6_Semester/Projekt_BBA/PingPong_Project/Data/processed_data/ttr_entwicklung_ohne_nall_erweitert.csv", index=False)

Zusammenführung abgeschlossen.
Gesamtzahl der Zeilen in TTR_Entwicklung_inport: 34379
Erfolgreich zugeordnete Zeilen: 34379 (100.00%)
Spalten im zusammengeführten DataFrame: 48
Anzahl der Duplikate nach InterneNr: 0
